In [ ]:
# ftp://ngs.sanger.ac.uk/production/ag1000g/phase1/AR3/variation/crosses/ar3/hdf5/ag1000g.crosses.phase1.ar3sites.3L.h5
# ftp://ngs.sanger.ac.uk/production/ag1000g/phase1/AR3/variation/crosses/ar3/hdf5/ag1000g.crosses.phase1.ar3sites.3R.h5


In [ ]:
import pickle
import gzip
import random

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import preprocessing 

%matplotlib inline

In [ ]:
mendelian_errors = pickle.load(gzip.open('mendelian_errors.pickle.gz', 'rb'))
feature_fit = np.load(gzip.open('feature_fit.npy.gz', 'rb'))
ordered_features = np.load(open('ordered_features', 'rb'))
num_features = len(ordered_features)

In [ ]:
total_observations = len(mendelian_errors)
error_observations = len(list(filter(lambda x: x[0] > 0,mendelian_errors.values())))
ok_observations = total_observations - error_observations
fraction_errors = error_observations/total_observations
print (total_observations, ok_observations, error_observations, 100*fraction_errors)
del mendelian_errors

In [ ]:
scaler = preprocessing.StandardScaler(copy=False)
scaler.fit(feature_fit[:,:-1])
scaler.transform(feature_fit[:,:-1])
#Original feature fit is now lost
np.save(gzip.open('scaled_fit.npy.gz', 'wb'), feature_fit, allow_pickle=False, fix_imports=False)

In [ ]:
feature_fit[:,9].max()

In [ ]:
feature_fit = np.load(gzip.open('feature_fit.npy.gz', 'rb'))

In [ ]:
#total_observations*fraction_errors
prob_ok_choice = error_observations / ok_observations

def accept_entry(row):
    if row[-1] == 1:
        return True
    return random.random() <= prob_ok_choice

accept_entry_v = np.vectorize(accept_entry, signature='(i)->()')

accepted_entries = accept_entry_v(feature_fit)

In [ ]:
balanced_fit = feature_fit[accepted_entries]
del feature_fit
balanced_fit.shape
len([x for x in balanced_fit if x[-1] == 1]), len([x for x in balanced_fit if x[-1] == 0])

In [ ]:
np.save(gzip.open('balanced_fit.npy.gz', 'wb'), balanced_fit, allow_pickle=False, fix_imports=False)

In [ ]:
[balanced_fit[:, column].mean() for column in range(num_features + 1)]

In [ ]:
#scaler = preprocessing.StandardScaler(copy=False) #Do not do this
#scaler.fit(balanced_fit[:,:-1])  # especially this!!!!
scaler.transform(balanced_fit[:,:-1])
#Original balanced fit is now lost
scaled_balanced_fit = balanced_fit # Better naming

In [ ]:
[scaled_balanced_fit[:, column].mean() for column in range(num_features + 1)]

In [ ]:
np.save(gzip.open('scaled_balanced_fit.npy.gz', 'wb'), scaled_balanced_fit, allow_pickle=False, fix_imports=False)

In [ ]:
scaled_balanced_fit.shape

In [ ]:
cv_indexes = np.random.random(scaled_balanced_fit.shape[0]) > 0.8

In [ ]:
scaled_balanced_fit_cv = balanced_fit[cv_indexes]
scaled_balanced_fit_100_test = balanced_fit[~cv_indexes]

In [ ]:
np.save(gzip.open('scaled_balanced_fit_cv.npy.gz', 'wb'), scaled_balanced_fit_cv, allow_pickle=False, fix_imports=False)
np.save(gzip.open('scaled_balanced_fit_100_test.npy.gz', 'wb'), scaled_balanced_fit_100_test, allow_pickle=False, fix_imports=False)

In [ ]:
np.save(gzip.open('scaled_balanced_fit_10_test.npy.gz', 'wb'), scaled_balanced_fit_100_test[
    np.random.random(scaled_balanced_fit_100_test.shape[0]) <= 0.1], allow_pickle=False, fix_imports=False)
np.save(gzip.open('scaled_balanced_fit_1_test.npy.gz', 'wb'), scaled_balanced_fit_100_test[
    np.random.random(scaled_balanced_fit_100_test.shape[0]) <= 0.01], allow_pickle=False, fix_imports=False)